In [1]:
import os
import random
import numpy as np
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.model_zoo import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import torch

# Register your training dataset with Detectron2
register_coco_instances("custom_dataset_train", {}, "train_output_coco_annotations.json", "train/images")

# Register your test dataset with Detectron2
register_coco_instances("custom_dataset_test", {}, "test_output_coco_annotations.json", "test/images")

# Define metadata for your dataset (class names)
metadata = MetadataCatalog.get("custom_dataset_train")

# Create a configuration
cfg = get_cfg()

# Set your custom configuration options here, for example:
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("custom_dataset_train",)
cfg.DATASETS.TEST = ("custom_dataset_test",)  # Include your test dataset
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000  # Increase the maximum number of iterations
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.DEVICE = 'cpu'  # Use 'cuda' if GPU is available

# Implement early stopping based on training accuracy
best_accuracy = 0.0
early_stopping_patience = 5  # Number of epochs to wait for improvement
best_model = None
for epoch in range(cfg.SOLVER.MAX_ITER):
    # Instantiate a trainer for each epoch
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)

    # Train the model for one epoch
    trainer.train()

    # Evaluate the model on the training set to check for accuracy improvement
    evaluator = COCOEvaluator("custom_dataset_train", cfg, False, output_dir="./output/")
    train_loader = build_detection_test_loader(cfg, "custom_dataset_train")
    train_results = inference_on_dataset(trainer.model, train_loader, evaluator)

    # Check if training accuracy has improved
    if train_results["bbox"]["AP"] > best_accuracy:
        best_accuracy = train_results["bbox"]["AP"]
        best_model = trainer.model.state_dict()  # Save the best model state_dict
    else:
        if epoch >= early_stopping_patience:
            print(f"Early stopping at epoch {epoch} as training accuracy did not improve.")
            break

# Save the best model state_dict to a file
if best_model is not None:
    best_model_path = "./best_model.pth"
    torch.save(best_model, best_model_path)
    print(f"Best model saved to {best_model_path}")

# Load the best model
if best_model is not None:
    cfg.MODEL.WEIGHTS = best_model_path
    predictor = DefaultPredictor(cfg)

    # Evaluate the best model on the test set
    evaluator = COCOEvaluator("custom_dataset_test", cfg, False, output_dir="./output/")
    test_loader = build_detection_test_loader(cfg, "custom_dataset_test")
    test_results = inference_on_dataset(predictor.model, test_loader, evaluator)

    # Show the test results
    print("Test results:")
    print(test_results)
else:
    print("No best model found.")


[09/08 03:08:32 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

[09/08 03:08:32 d2.engine.train_loop]: Starting training from iteration 0


/Users/dhruvraj/venv-metal/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[09/08 03:09:18 d2.utils.events]:  eta: 0:00:00  iter: 9  total_loss: 2.626  loss_cls: 0.6691  loss_box_reg: 0.2811  loss_mask: 0.687  loss_rpn_cls: 0.8792  loss_rpn_loc: 0.2355    time: 4.2905  last_time: 4.4539  data_time: 0.1551  last_data_time: 0.0029   lr: 0.00022503  
[09/08 03:09:22 d2.engine.hooks]: Overall training speed: 8 iterations in 0:00:34 (4.2905 s / it)
[09/08 03:09:22 d2.engine.hooks]: Total training time: 0:00:38 (0:00:04 on hooks)
[09/08 03:09:22 d2.data.datasets.coco]: Loaded 10 images in COCO format from test_output_coco_annotations.json
[09/08 03:09:22 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[09/08 03:09:22 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[09/08 03:09:22 d2.data.common]: Serializing 10 elements to byte tensors and concatenating them all ...
[09/08 03:09:22 d2.data.common]: 

...

[09/08 03:10:32 d2.engine.train_loop]: Starting training from iteration 0
[09/08 03:11:20 d2.utils.events]:  eta: 0:00:00  iter: 9  total_loss: 2.622  loss_cls: 0.73  loss_box_reg: 0.2856  loss_mask: 0.6897  loss_rpn_cls: 0.7346  loss_rpn_loc: 0.2188    time: 4.5742  last_time: 4.1488  data_time: 0.1446  last_data_time: 0.0015   lr: 0.00022503  
[09/08 03:11:20 d2.engine.hooks]: Overall training speed: 8 iterations in 0:00:36 (4.5742 s / it)
[09/08 03:11:20 d2.engine.hooks]: Total training time: 0:00:36 (0:00:00 on hooks)
[09/08 03:11:20 d2.data.datasets.coco]: Loaded 10 images in COCO format from test_output_coco_annotations.json
[09/08 03:11:20 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[09/08 03:11:20 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[09/08 03:11:20 d2.data.common]: Serializing 10 elements to byte

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

[09/08 03:11:31 d2.engine.train_loop]: Starting training from iteration 0
[09/08 03:12:19 d2.utils.events]:  eta: 0:00:00  iter: 9  total_loss: 2.724  loss_cls: 0.7477  loss_box_reg: 0.2941  loss_mask: 0.69  loss_rpn_cls: 0.8521  loss_rpn_loc: 0.2202    time: 4.3654  last_time: 4.6538  data_time: 0.1432  last_data_time: 0.0026   lr: 0.00022503  
[09/08 03:12:19 d2.engine.hooks]: Overall training speed: 8 iterations in 0:00:34 (4.3655 s / it)
[09/08 03:12:19 d2.engine.hooks]: Total training time: 0:00:35 (0:00:00 on hooks)
[09/08 03:12:19 d2.data.datasets.coco]: Loaded 10 images in COCO format from test_output_coco_annotations.json
[09/08 03:12:19 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[09/08 03:12:19 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[09/08 03:12:19 d2.data.common]: Serializing 10 elements to byte

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

[09/08 03:12:29 d2.engine.train_loop]: Starting training from iteration 0


In [ ]:
import os
import random
import numpy as np
from detectron2.data import MetadataCatalog
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
import cv2

# Load the saved model weights and configuration
cfg = get_cfg()
cfg.merge_from_file("model_config.yaml")  # Replace with the path to your model's configuration file
cfg.MODEL.WEIGHTS = "best_model.pth"  # Replace with the path to your saved model weights

# Set the device to CPU
cfg.MODEL.DEVICE = 'cpu'

# Create a predictor using the loaded model
predictor = DefaultPredictor(cfg)

# Get a random image from the "train/images" folder
image_folder = 'train/images'  # Replace with the path to your image folder
image_files = os.listdir(image_folder)
random_image_file = random.choice(image_files)
image_path = os.path.join(image_folder, random_image_file)

# Read the random image
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Get predictions on the image
outputs = predictor(image)

# Filter predictions with confidence > 75%
instances = outputs["instances"]
filtered_instances = instances[instances.scores > 0.75]

# Visualize the original image with filtered predictions
v = Visualizer(image, metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(filtered_instances.to("cpu"))

# Show the image with predictions
cv2.imshow("Predictions", out.get_image()[:, :, ::-1])

# Construct the path to the corresponding mask image
mask_folder = 'train/binned_targets'  # Replace with the path to your mask folder
mask_file = os.path.join(mask_folder, os.path.splitext(random_image_file)[0] + '_target.png')

# Read and show the mask image
mask_image = cv2.imread(mask_file)
cv2.imshow("Mask", mask_image)

cv2.waitKey(0)
# Close all OpenCV windows when any key is pressed
cv2.destroyAllWindows()
cv2.waitKey(1)